In [3]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)
import asyncio
from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama
from llama_index.utils.workflow import draw_all_possible_flows

In [2]:
class FirstEvent(Event):
    first_output: str


class SecondEvent(Event):
    second_output: str
    response: str


class ProgressEvent(Event):
    msg: str

In [4]:
class MyWorkflow(Workflow):
    @step
    async def step_one(self, ctx: Context, ev: StartEvent) -> FirstEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step one is happening"))
        return FirstEvent(first_output="First step complete.")

    @step
    async def step_two(self, ctx: Context, ev: FirstEvent) -> SecondEvent:
        # llm = OpenAI(model="gpt-4o-mini")
        llm = Ollama(model="qwen2.5:7b", request_timeout=720.0)
        generator = await llm.astream_complete(
            "Please give me the first 3 paragraphs of Moby Dick, a book in the public domain."
        )
        async for response in generator:
            # Allow the workflow to stream this piece of response
            ctx.write_event_to_stream(ProgressEvent(msg=response.delta))
        return SecondEvent(
            second_output="Second step complete, full response attached",
            response=str(response),
        )

    @step
    async def step_three(self, ctx: Context, ev: SecondEvent) -> StopEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step three is happening"))
        return StopEvent(result="Workflow complete.")

In [7]:
w = MyWorkflow(timeout=720, verbose=True)
handler = w.run(first_input="Start the workflow.")

async for ev in handler.stream_events():
    if isinstance(ev, ProgressEvent):
            print(ev.msg)

final_result = await handler
print("Final result", final_result)

draw_all_possible_flows(MyWorkflow, filename="streaming_workflow.html")

Running step step_one
Step step_one produced event FirstEvent
Running step step_two
Step one is happening
Certainly
!
 Here
 are
 the
 first
 three
 paragraphs
 from
 Herman
 Mel
ville
's
 "
M
oby
-D
ick
":


Call
 me
 Ish
ma
el
.
 Some
 years
 ago
—
never
 mind
 how
 long
 precisely
—
having
 little
 or
 no
 money
 in
 my
 purse
,
 and
 nothing
 particular
 to
 interest
 me
 on
 shore
,
 I
 thought
 I
 would
 sail
 about
 a
 little
 and
 see
 the
 wat
ery
 part
 of
 the
 world
.
 It
 is
 a
 way
 I
 have
 of
 driving
 off
 the
 sple
en
 and
 regulating
 the
 circulation
.
 Whenever
 I
 find
 myself
 growing
 grim
 over
 the
 countless
 human
 stupidity
,
 I
 remember
 again
 my
 father
's
 maxim
:
 "
Keep
 your
 temper
,
 Ish
ma
el
,
 and
 don
't
 kill
 no
 whales
 neither
."


There
 are
 times
 when
 I
 am
 almost
 inclined
 to
 think
 that
 two
 out
 of
 every
 three
 men
 in
 this
 world
 are
 women
;
 for
 so
 many
 of
 the
 other
 half
 are
 weak
 as
 women
.
 Not
,
 indeed
,
 th

/Users/hyjiang/miniforge3/envs/py311/lib/python3.11/site-packages/networkx/utils/backends.py:453: RuntimeWarning: coroutine 'main' was never awaited
  self = object.__new__(cls)


<class 'NoneType'>
<class '__main__.FirstEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.SecondEvent'>
streaming_workflow.html


python(18967) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
